<a href="https://colab.research.google.com/github/AiAnonymousPT/Run-LLMs-Locally/blob/main/Quantize-llamacpp/Quantize_llamacpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantize a model with Llama.cpp

Sources
- https://github.com/ggerganov/llama.cpp/discussions/2948
- https://hackernoon.com/quantizing-large-language-models-with-llamacpp-a-clean-guide-for-2024

In [ ]:
pip install huggingface_hub

### Compile Llama.cpp

In [1]:
!git clone https://github.com/ggerganov/llama.cpp

# compile with CUDA support, modify to run on Apple Silicon
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 20407, done.
remote: Counting objects: 100% (6849/6849), done.
remote: Compressing objects: 100% (484/484), done.
remote: Total 20407 (delta 6621), reused 6434 (delta 6362), pack-reused 13558
Receiving objects: 100% (20407/20407), 22.27 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (14420/14420), done.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcas

### Download model from HF

In [2]:
from huggingface_hub import snapshot_download

model_name = "cognitivecomputations/dolphin-2.6-mistral-7b-dpo-laser"
base_model = "./original_model/"
snapshot_download(repo_id=model_name, local_dir=base_model, local_dir_use_symlinks=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.13k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

'/content/original_model'

In [3]:
# !mkdir ./quantized_model/
# !python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model: original_model
Traceback (most recent call last):
  File "/content/llama.cpp/convert-hf-to-gguf.py", line 2056, in <module>
    main()
  File "/content/llama.cpp/convert-hf-to-gguf.py", line 2036, in main
    model_class = Model.from_model_architecture(hparams["architectures"][0])
  File "/content/llama.cpp/convert-hf-to-gguf.py", line 204, in from_model_architecture
    raise NotImplementedError(f'Architecture {arch!r} not supported!') from None
NotImplementedError: Architecture 'MistralForCausalLM' not supported!


### Convert to FP16 GGUF

In [4]:
!mkdir ./quantized_model/
!python llama.cpp/convert.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model file original_model/model-00001-of-00003.safetensors
Loading model file original_model/model-00001-of-00003.safetensors
Loading model file original_model/model-00002-of-00003.safetensors
Loading model file original_model/model-00003-of-00003.safetensors
params = Params(n_vocab=32001, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('original_model'))
Found vocab files: {'spm': PosixPath('original_model/tokenizer.model'), 'bpe': None, 'hfft': PosixPath('original_model/tokenizer.json')}
Loading vocab file PosixPath('original_model/tokenizer.model'), type 'spm'
Vocab info: <SentencePieceVocab with 32000 base tokens and 1 added tokens>
Special vocab info: <SpecialVocab with 0 merges, special tokens {'bos': 1, 'eos': 2, 'unk': 0, 'pad': 2

### Quantize the model at Q4

In [5]:
import os

methods = ["q4_k_m"]
quantized_path = "./quantized_model/"

for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

### Test the model

In [6]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User: " -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 2427 (044ec4b2)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1710433362
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                   

### Push GGUF to HuggingFace

In [17]:
from huggingface_hub import login
login()

In [18]:
from huggingface_hub import HfApi, create_repo, upload_file

model_path = "./quantized_model/Q4_K_M.gguf"
repo_name = "dolphin-2.6-mistral-7b-dpo-laser-GGUF-quantized"
repo_url = create_repo(repo_name, private=False)

api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id="mswaringen/dolphin-2.6-mistral-7b-dpo-laser-GGUF-quantized",
    repo_type="model",
)

Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mswaringen/dolphin-2.6-mistral-7b-dpo-laser-GGUF-quantized/commit/6a2b5bc7a4ff5bfce490ffcd99ea070603d8243e', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='6a2b5bc7a4ff5bfce490ffcd99ea070603d8243e', pr_url=None, pr_revision=None, pr_num=None)